# Семинар 15

## Частотные списки

В этом семинаре мы познакомимся с некоторыми простыми инструментами для лексического анализа текста. Один из наиболее простых способов проанализировать содержание текста — это выделить наиболее частотные слова. Список всех слов текста, в котором каждому слову соответствует его частотность, называется **частотным списком**.

Обратите внимание: термин «частотный список» как будто предполагает, что каждому слову соответствует не количество вхождений слова в тексте, а частота слова. Например, в тексте «быть или не быть» у слова *быть* встречается с частотой 50% (потому что 50% токенов — употребления этого слова). В реальности, однако, термином «частотный список» обычно называют и такие списки, в которых собрано абсолютное количество вхождений слова (например, $2$ для слова *быть*, которое встречается дважды).

Мы уже умеем делать частотные списки с помощью обычного питоновского словаря — нужно пройтись по всему списку токенов и за каждое употребление слова увеличить значение в словаре на единичку:

In [1]:
from nltk import word_tokenize

text = "Быть или не быть?"
tokens = word_tokenize(text.lower())

freq_dict = {}
for token in tokens:
    if not token in freq_dict:
        freq_dict[token] = 0
    freq_dict[token] += 1

print(freq_dict)

{'быть': 2, 'или': 1, 'не': 1, '?': 1}


Теперь же мы познакомимся с более простым инструментом, который уже реализует весь этот алгоритм за нас (то есть «счётчик»). Это новый тип данных (а точнее, так называемый «подкласс» словарей) — обычный словарь, но с некоторыми фишками для дополнительного удобства. Этот тип под названием **`Counter`** можно найти в библиотеке `collections` (см. [документацию](https://docs.python.org/3/library/collections.html)). Чтобы создать объект типа `Counter`, нужно вызвать одноимённую функцию и подать в неё итерируемый объект (например, список). `Counter` сам подсчитает, каких элементов сколько:

In [2]:
from collections import Counter

In [3]:
Counter(tokens)

Counter({'быть': 2, 'или': 1, 'не': 1, '?': 1})

В `Counter` можно подать не только список, а вообще любой итерируемый объект — например, если нужно, можно подать строку:

In [4]:
Counter("длинношеее")

Counter({'е': 3, 'н': 2, 'д': 1, 'л': 1, 'и': 1, 'о': 1, 'ш': 1})

Обратите внимание на важное отличие от обычного словаря: `Counter` не просто считает всё сам, но ещё и сортирует ключи в порядке убывания значений! То есть первыми в таком словаре всегда будут наиболее частотные элементы. Это очень удобно, особенно когда элементов очень много — не нужно проводить дополнительную сортировку или искать наиболее частотные элементы вручную, `Counter` делает это автоматически.

Более того, `Counter` отличается от обычного словаря тем, что у него есть специальные методы для ещё большего удобства. К примеру, метод **`.most_common()`** возвращает топ-N самых частотных элементов (задать число N можно с помощью аргумента `n`). Этот метод вернёт список с кортежами, в каждом из которых — набор из элемента и его частотности:

In [5]:
Counter("длинношеее").most_common(n=2)   # два самых частотных элемента

[('е', 3), ('н', 2)]

### Выделение частотных слов в тексте

Воспользуемся полученными на последних семинарах знаниями об автоматической обработке текста вместе с `Counter`, чтобы в несколько простых шагов извлечь из текста самые частотные слова. В качестве примера будем работать с текстом книги «Маленький принц» Антуана де Сент-Экзюпери (**[ссылка](https://github.com/maxmerben/hse-python-assyr-uva-2025/blob/main/other/sem15-little_prince.txt)**):

In [6]:
with open("sem15-little_prince.txt", encoding="utf-8") as f:
    text = f.read()

Перед непосредственно составлением частотного списка нам необходимо провести **предварительную обработку текста**. Этот этап включает в себя подготовку и очистку текста. Какие конкретно шаги нужно предпринять для обработки, зависит от конкретной задачи, которую вы решаете. Например, в этот этап могут входить: приведение к одному регистру, токенизация, очистка текста от стоп-слов / знаков препинания / чисел, лемматизация / стемминг. Здесь для примера реализуем минимальный набор: приведение к строчному регистру, токенизация, очистка от стоп-слов, лемматизация. Заранее импортируем все нужные модули и создадим объект-морфоанализатор:

In [7]:
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import pymorphy3

my_analyzer = pymorphy3.MorphAnalyzer()

Токенизируем текст с помощью `nltk.word_tokenize`:

In [8]:
tokens = word_tokenize(text.lower())
print(tokens[:50])

['антуан', 'де', 'сент-экзюпери', 'маленький', 'принц', 'леону', 'верту', 'прошу', 'детей', 'простить', 'меня', 'за', 'то', ',', 'что', 'я', 'посвятил', 'эту', 'книжку', 'взрослому', '.', 'скажу', 'в', 'оправдание', ':', 'этот', 'взрослый', '—', 'мой', 'самый', 'лучший', 'друг', '.', 'и', 'еще', ':', 'он', 'понимает', 'все', 'на', 'свете', ',', 'даже', 'детские', 'книжки', '.', 'и', ',', 'наконец', ',']


Теперь удалим из этого списка токенов все стоп-слова — воспользуемся списком стоп-слов для русского из `nltk.corpus.stopwords`:

In [9]:
clean_tokens = []

for token in tokens:
    if token not in stopwords.words("russian"):
        clean_tokens.append(token)

print(clean_tokens[:50])

['антуан', 'де', 'сент-экзюпери', 'маленький', 'принц', 'леону', 'верту', 'прошу', 'детей', 'простить', ',', 'посвятил', 'книжку', 'взрослому', '.', 'скажу', 'оправдание', ':', 'взрослый', '—', 'самый', 'лучший', 'друг', '.', ':', 'понимает', 'свете', ',', 'детские', 'книжки', '.', ',', ',', 'живет', 'франции', ',', 'голодно', 'холодно', '.', 'очень', 'нуждается', 'утешении', '.', 'это', 'оправдывает', ',', 'посвящу', 'книжку', 'тому', 'мальчику']


На этом этапе было бы полезно также удалить из списка токенов все знаки препинания — попробуйте сделать это самостоятельно!

Наконец, проведём лемматизацию очищенного списка токенов с помощью `pymorphy3`. Для этого для каждого токена сначала получим наиболее вероятный разбор, а потом получим лемму (начальную форму слова) методом `.normal_form`. Получившиеся леммы запишем в тот же список `clean_tokens` на место исходных токенов:

In [10]:
for i in range(len(clean_tokens)):
    token_parse = my_analyzer.parse(clean_tokens[i])[0]   # <-- разбор, объект типа Parse
    clean_tokens[i] = token_parse.normal_form             # <-- кладём на место токена его лемму

print(clean_tokens[:50])

['антуан', 'де', 'сент-экзюпери', 'маленький', 'принц', 'леон', 'верот', 'просить', 'ребёнок', 'простить', ',', 'посвятить', 'книжка', 'взрослый', '.', 'сказать', 'оправдание', ':', 'взрослый', '—', 'самый', 'хороший', 'друг', '.', ':', 'понимать', 'свет', ',', 'детский', 'книжка', '.', ',', ',', 'жить', 'франция', ',', 'голодный', 'холодно', '.', 'очень', 'нуждаться', 'утешение', '.', 'это', 'оправдывать', ',', 'посвятить', 'книжка', 'тот', 'мальчик']


Ура, предварительная обработка завершена! Теперь можно положить лемматизированные токены в `Counter`, и мы получим частотный список по «Маленькому принцу»:

In [11]:
freq_dict = Counter(clean_tokens)
freq_dict

Counter({',': 1267,
         '.': 1117,
         '—': 898,
         'маленький': 202,
         'принц': 179,
         '?': 165,
         ':': 149,
         'это': 135,
         'сказать': 134,
         '!': 122,
         'очень': 74,
         'планета': 65,
         'человек': 51,
         'цветок': 49,
         'звезда': 47,
         '«': 46,
         '»': 46,
         'мой': 42,
         'ты': 39,
         'спросить': 38,
         'знать': 38,
         'один': 38,
         'день': 38,
         'такой': 37,
         'барашек': 37,
         'стать': 36,
         'который': 35,
         'король': 34,
         'свой': 33,
         'друг': 31,
         'ответить': 31,
         'взрослый': 29,
         'мочь': 28,
         'год': 24,
         'говорить': 24,
         'лис': 24,
         'другой': 23,
         'нарисовать': 22,
         'хотеть': 22,
         'твой': 22,
         'увидеть': 21,
         'добрый': 21,
         'географ': 21,
         'видеть': 20,
         'подумать': 20,
  

In [12]:
freq_dict.most_common(20)

[(',', 1267),
 ('.', 1117),
 ('—', 898),
 ('маленький', 202),
 ('принц', 179),
 ('?', 165),
 (':', 149),
 ('это', 135),
 ('сказать', 134),
 ('!', 122),
 ('очень', 74),
 ('планета', 65),
 ('человек', 51),
 ('цветок', 49),
 ('звезда', 47),
 ('«', 46),
 ('»', 46),
 ('мой', 42),
 ('ты', 39),
 ('спросить', 38)]

(Видим, что на первом месте с большим отрывом — знаки препинания. Чтобы в списке остались только полноценные русские слова, нам нужно на каком-то из предыдущих этапов отсеять знаки препинания. Это можно сделать самыми разными способами — например, регулярками.)

Итак, мы научились вычленять из русского текста самые частотные слова с помощью `nltk`, `pymorphy3` и `collections.Counter`! Напоследок посмотрим на то, как весь этот пайплайн можно было бы реализовать более компактно — в одном большом цикле:

In [13]:
lemmatized_tokens = []

for token in word_tokenize(text.lower()):
    if token not in stopwords.words("russian"):
        token_parse = my_analyzer.parse(token)[0]
        lemmatized_tokens.append(token_parse.normal_form)

In [14]:
Counter(lemmatized_tokens).most_common(10)

[(',', 1267),
 ('.', 1117),
 ('—', 898),
 ('маленький', 202),
 ('принц', 179),
 ('?', 165),
 (':', 149),
 ('это', 135),
 ('сказать', 134),
 ('!', 122)]

### *N*-граммы

Напоследок рассмотрим ещё один полезный инструмент для вычленения частотных элементов в тексте — **_n_-граммы**. _N_-граммы — это все последовательные сочетания слов в тексте. Например, в тексте «быть или не быть» есть такие **биграммы**: «быть или», «или не», «не быть». А ещё такие **триграммы**: «быть или не», «или не быть». На таком игрушечном тексте вычленение _n_-грамм не имеет смысла, но на большом объёме данных это может быть полезно для поиска частотных словосочетаний, устойчивых выражений, повторяющихся оборотов (возможно, говорящих что-то про содержание текста).

В питоне _n_-граммы легко получить с помощью специальной функции **`ngrams()`** из модуля `nltk.util`:

In [15]:
from nltk.util import ngrams

Засунем в эту функцию наш текст и укажем в аргументе `n` количество слов в словосочетании (чтобы получить *би*грамму, *три*грамму и так далее). Мы получим особый итерируемый объект, который можно превратить в список функцией `list()`:

In [16]:
list(ngrams(clean_tokens, 2))

[('антуан', 'де'),
 ('де', 'сент-экзюпери'),
 ('сент-экзюпери', 'маленький'),
 ('маленький', 'принц'),
 ('принц', 'леон'),
 ('леон', 'верот'),
 ('верот', 'просить'),
 ('просить', 'ребёнок'),
 ('ребёнок', 'простить'),
 ('простить', ','),
 (',', 'посвятить'),
 ('посвятить', 'книжка'),
 ('книжка', 'взрослый'),
 ('взрослый', '.'),
 ('.', 'сказать'),
 ('сказать', 'оправдание'),
 ('оправдание', ':'),
 (':', 'взрослый'),
 ('взрослый', '—'),
 ('—', 'самый'),
 ('самый', 'хороший'),
 ('хороший', 'друг'),
 ('друг', '.'),
 ('.', ':'),
 (':', 'понимать'),
 ('понимать', 'свет'),
 ('свет', ','),
 (',', 'детский'),
 ('детский', 'книжка'),
 ('книжка', '.'),
 ('.', ','),
 (',', ','),
 (',', 'жить'),
 ('жить', 'франция'),
 ('франция', ','),
 (',', 'голодный'),
 ('голодный', 'холодно'),
 ('холодно', '.'),
 ('.', 'очень'),
 ('очень', 'нуждаться'),
 ('нуждаться', 'утешение'),
 ('утешение', '.'),
 ('.', 'это'),
 ('это', 'оправдывать'),
 ('оправдывать', ','),
 (',', 'посвятить'),
 ('посвятить', 'книжка'),
 ('

Если теперь в `Counter` послать не список токенов, а список _n_-грамм, то мы узнаем, какие двусловные выражения чаще всего повторяются. Впрочем, результат ожидаемый:

In [17]:
Counter(ngrams(clean_tokens, 2)).most_common(10)

[(('.', '—'), 339),
 ((',', '—'), 192),
 (('маленький', 'принц'), 178),
 (('?', '—'), 106),
 (('—', 'сказать'), 92),
 ((':', '—'), 78),
 (('принц', '.'), 73),
 (('—', ','), 57),
 (('.', 'это'), 40),
 (('!', '—'), 40)]

Поэкспериментируем с размером _n_-грамм:

In [18]:
Counter(ngrams(clean_tokens, 3)).most_common(10)

[((',', '—', 'сказать'), 81),
 (('маленький', 'принц', '.'), 73),
 (('принц', '.', '—'), 56),
 (('.', 'маленький', 'принц'), 32),
 (('сказать', 'маленький', 'принц'), 28),
 (('—', 'сказать', 'маленький'), 27),
 (('маленький', 'принц', ','), 25),
 (('.', '—', ','), 22),
 (('.', '—', 'это'), 21),
 (('?', '—', 'спросить'), 20)]

In [19]:
Counter(ngrams(clean_tokens, 4)).most_common(10)

[(('маленький', 'принц', '.', '—'), 56),
 (('—', 'сказать', 'маленький', 'принц'), 27),
 ((',', '—', 'сказать', 'маленький'), 25),
 (('сказать', 'маленький', 'принц', '.'), 20),
 (('?', '—', 'спросить', 'маленький'), 16),
 (('—', 'спросить', 'маленький', 'принц'), 16),
 (('спросить', 'маленький', 'принц', '.'), 15),
 (('—', 'сказать', '.', '—'), 13),
 ((',', '—', 'сказать', '.'), 12),
 (('.', '—', 'добрый', 'день'), 10)]

Набор частотных _n_-грамм может сильно отличаться в зависимости от типа и жанра исследуемого текста. Попробуем взять радикально другой текст — например, «Манифест коммунистической партии» Карла Маркса и Фридриха Энгельса (**[ссылка](https://raw.githubusercontent.com/kirs53/datasets/refs/heads/main/Manifesto%20of%20the%20Communist%20Party.txt)**) — публицистический текст социально-политической тематики. В нём в числе частотных биграмм будут различные устойчивые выражения из соответствующей области — «наёмный труд», «рабочий класс», «буржуазное общество», «частная собственность» и прочие:

In [20]:
with open("Manifesto of the Communist Party.txt", "r", encoding="utf-8") as f:
    manifesto = f.read()

lemmatized_manifesto = []

for token in word_tokenize(manifesto.lower()):
    if token not in stopwords.words("russian") and token not in ".,:;?!()[]-–—«»":
        token_parse = my_analyzer.parse(token)[0]
        lemmatized_manifesto.append(token_parse.normal_form)

freq_bigrams = Counter(ngrams(lemmatized_manifesto, 2))
freq_bigrams.most_common(20)

[(('английский', 'издание'), 41),
 (('издание', '1888'), 40),
 (('1888', 'г.'), 25),
 (('примечание', 'энгельс'), 24),
 (('свой', 'собственный'), 16),
 (('энгельс', 'английский'), 16),
 (('1888', 'г'), 16),
 (('буржуазный', 'общество'), 14),
 (('немецкий', 'издание'), 14),
 (('производственный', 'отношение'), 14),
 (('т.', 'е.'), 14),
 (('г.', 'вместо'), 14),
 (('наёмный', 'труд'), 13),
 (('господствующий', 'класс'), 13),
 (('такой', 'образ'), 12),
 (('отношение', 'собственность'), 12),
 (('рабочий', 'класс'), 12),
 (('вместо', 'слово'), 12),
 (('условие', 'жизнь'), 11),
 (('частный', 'собственность'), 10)]

При этом, как вы можете видеть, есть здесь и совсем обычные словосочетания, которые могут встретиться как в программном политическом манифесте, так и в фанфике, в SMM-тексте в социальных сетях или в энцилклопедии — например, «свой собственный» или «такой образ». Эти биграммы действительно частотны в «Манифесте», но они также частотны в других типах текстов, поэтому они, в отличие от выражений типа «наёмный труд», не сообщают нам ничего полезного об этом тексте, не выделяют его среди прочих. Это одна из проблем с частотными списками и _n_-граммами — даже после удаления стоп-слов в них всё равно много «мусора», который бесполезен для задач выделения ключевых слов или классификации текста по темам. Чтобы по-настоящему заниматься этими задачами, необходимо сравнивать разные типы текстов между собой. Этим занимается чуть более продвинутый алгоритм выделения ключевых слов, который называется ***TF-IDF***. К сожалению, мы не проходим *TF-IDF* в этом курсе, но если вам интересно узнать про него, рекомендую почитать про него в интернете (например, [вот](https://www.learndatasci.com/glossary/tf-idf-term-frequency-inverse-document-frequency/)).

Всё!…